In [1]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

import pytorch_lightning as pl

In [2]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [6]:
class BRNN(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out
    def prepare_data(self):
        self.train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

        self.test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)   
    def train_dataloader(self):
        return DataLoader(dataset=self.train_dataset, batch_size=batch_size, shuffle=True)

    def val_dataloader(self):
        return self.train_dataloader()

    def test_dataloader(self):
        return DataLoader(dataset=self.test_dataset, batch_size=batch_size, shuffle=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=learning_rate)

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        x = x.squeeze(1)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        x = x.squeeze(1)
        y_hat = model(x)
        loss = F.cross_entropy(y_hat, y)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        x = x.squeeze(1)
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}


In [7]:
model = BRNN(input_size, hidden_size, num_layers, num_classes)   
# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus = 1,max_epochs=5)    
trainer.fit(model)

17it/s]
Epoch 4:  50%|█████     | 938/1876 [01:01<01:01, 15.35it/s, loss=0.047, v_num=29]
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 1876/1876 [01:23<00:00, 22.37it/s, loss=0.047, v_num=29]


1

In [8]:
trainer.test()

Testing:  99%|█████████▊| 155/157 [00:03<00:00, 37.99it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(0.0489, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 157/157 [00:03<00:00, 44.39it/s]


[{'test_loss': 0.0489131435751915}]